## Setup

In [2]:
import sys
from pathlib import Path

from aocd import get_data, submit

current_day is only available in December (EST)


In [3]:
# Add parent directory to path to allow relative imports into Jupyter notebook
sys.path.append(str(Path.cwd().parent))

In [207]:
# Get raw advent-of-code data
data: str = get_data(year=2024, day=17)

## Part a

In [142]:
# Imports
import re

In [143]:
# Constants
COMBO_OPS = {  # Mapping of combo operands to register values
    4: "A",
    5: "B",
    6: "C",
}

In [194]:
# Functions
def get_combo_op_value(operand: int, registers: dict) -> int:
    """Get the value of a combo operand."""
    if operand not in range(7):
        err_msg = f"Operand {operand} must be between 0 and 6"
        raise ValueError(err_msg)
    if operand <= 3:
        return operand
    return registers[COMBO_OPS[operand]]


def adv(registers: dict, combo_op: int) -> None:
    """Divide the A register by 2^combo_op, truncate the result, and store the result in A."""
    combo_op_value = get_combo_op_value(combo_op, registers)
    numerator = registers["A"]
    denominator = 2**combo_op_value
    registers["A"] = numerator // denominator


def bxl(registers: dict, literal_op: int) -> None:
    """Bitwise XOR the B register with literal_op and store the result in B."""
    registers["B"] ^= literal_op


def bst(registers: dict, combo_op: int) -> None:
    """Find the value of the combo operand modulo 8 and store it in the B register."""
    combo_op_value = get_combo_op_value(combo_op, registers)
    registers["B"] = combo_op_value % 8


def jnz(registers: dict, literal_op: int) -> int | None:
    """Jump to the instruction at the value of the literal_op if register A is not zero."""
    if registers["A"] != 0:
        return literal_op
    return None


def bxc(registers: dict, _: int) -> None:
    """Bitwise XOR the B register with the C register and store the result in B."""
    registers["B"] ^= registers["C"]


def out(registers: dict, combo_op: int) -> int:
    """Return the value of the combo operand."""
    combo_op_value = get_combo_op_value(combo_op, registers)
    return combo_op_value % 8


def bdv(registers: dict, combo_op: int) -> None:
    """Divide the A register by 2^combo_op, truncate the result, and store the result in B."""
    combo_op_value = get_combo_op_value(combo_op, registers)
    numerator = registers["A"]
    denominator = 2**combo_op_value
    registers["B"] = numerator // denominator


def cdv(registers: dict, combo_op: int) -> None:
    """Divide the A register by 2^combo_op, truncate the result, and store the result in C."""
    combo_op_value = get_combo_op_value(combo_op, registers)
    numerator = registers["A"]
    denominator = 2**combo_op_value
    registers["C"] = numerator // denominator


OP_CODES = {  # Mapping of instruction opcodes to their functions
    0: adv,
    1: bxl,
    2: bst,
    3: jnz,
    4: bxc,
    5: out,
    6: bdv,
    7: cdv,
}


def run_program(registers: dict[str, int], program: list[int]) -> str:
    """Run the program on the registers and return the output."""
    instruction_pointer = 0
    outputs = []

    while instruction_pointer < len(program):
        op_code, operand = program[instruction_pointer : instruction_pointer + 2]
        op = OP_CODES[op_code]

        output = op(registers, operand)

        if op_code == 3 and output is not None:  # Jump to new instruction
            instruction_pointer = output
            continue  # Skip incrementing instruction_pointer
        if op_code == 5:  # Store output
            outputs.append(output)

        instruction_pointer += 2

    return ",".join(map(str, outputs))

In [208]:
# Split data into registers and operations
registers_str, ops_str = data.split("\n\n")

registers = {
    match.group(1): int(match.group(2))
    for line in registers_str.splitlines()
    if (match := re.fullmatch(r"Register ([ABC]): (\d+)", line))
}

operations = (
    [int(x) for x in match.group(1).split(",")] if (match := re.fullmatch(r"Program: (\d(?:,\d)+)", ops_str)) else []
)

In [196]:
# Run the program
outputs = run_program(registers, operations)

In [ ]:
# Submit answer
submit(outputs, part="a", day=17, year=2024)

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## Part b

In [204]:
def check_program(initial_a: int, program: list[int]) -> tuple[bool, list[int]]:
    """Run program with given A value, return if outputs match program."""
    registers = {"A": initial_a, "B": 0, "C": 0}
    instruction_pointer = 0
    outputs = []

    while instruction_pointer < len(program):
        op_code, operand = program[instruction_pointer : instruction_pointer + 2]
        output = OP_CODES[op_code](registers, operand)

        if op_code == 3:  # jnz
            if registers["A"] != 0:
                instruction_pointer = operand
                continue
        elif op_code == 5:  # out
            outputs.append(output)
            if len(outputs) > len(program):  # Early exit if too long
                return False, outputs
            if outputs[-1] != program[len(outputs) - 1]:  # Early exit if mismatch
                return False, outputs

        instruction_pointer += 2

    return len(outputs) == len(program) and outputs == program, outputs

In [213]:
def find_self_reproducing_value(program: list[int]) -> int:
    """Find the value of A that will produce the program as output."""

    def run_with_a(a: int) -> list[int]:
        registers = {"A": a, "B": 0, "C": 0}
        ip = 0
        outputs = []

        while ip < len(program):
            op_code, operand = program[ip : ip + 2]
            output = OP_CODES[op_code](registers, operand)

            if op_code == 3:
                if registers["A"] != 0:
                    ip = operand
                    continue
            elif op_code == 5:
                outputs.append(output)
            ip += 2

        return outputs

    # Start with high power of 8 to ensure enough digits (16)
    a = 8**15
    power = 14
    matched = program[-1:]  # Start matching last digit

    while True:
        outputs = run_with_a(a)

        if outputs == program:
            return a

        if outputs[-len(matched) :] == matched:
            power = max(0, power - 1)  # Decrease power when match found
            matched = program[-(len(matched) + 1) :]  # Extend match target
        else:
            a += 8**power  # Try next value at current power

In [214]:
quine_a = find_self_reproducing_value(operations)

In [215]:
# Submit answer
submit(quine_a, part="b", day=17, year=2024)

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 17! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
